In [ ]:
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Add,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D,AveragePooling2D,MaxPooling2D
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.initializers import glorot_uniform
 

In [19]:
def identity_block(x,filter_size,number_of_filters):
  x_shortcut=x
  F1,F2,F3=number_of_filters
  
  ### 1st layer
  x=Conv2D(filters=F1,kernel_size=(1,1),strides=(1,1),padding='valid')(x)
  x=BatchNormalization(axis=3)(x)
  x=Activation('relu')(x)

  ### 2nd layer
  x=Conv2D(filters=F2,kernel_size=(filter_size,filter_size),strides=(1,1),padding='same')(x)
  x=BatchNormalization(axis=3)(x)
  x=Activation('relu')(x)

  ### 3rd layer
  x=Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid')(x)
  x=BatchNormalization(axis=3)(x)

  ### Final layer adding x with x_shortcut
  x=Add()([x,x_shortcut])
  x=Activation('relu')(x)

  return x



In [20]:
def convolutional_block(x,f,filters,s=2):
  x_shortcut=x
  F1,F2,F3=filters

  ## 1st layer
  x=Conv2D(F1,(1,1),(s,s))(x)
  x=BatchNormalization(axis=3)(x)
  x=Activation('relu')(x)

  ## 2nd layer
  x=Conv2D(F2,(f,f),(1,1),padding='same')(x)
  x=BatchNormalization(axis=3)(x)
  x=Activation('relu')(x)

  ### 3rd layer
  x=Conv2D(F3,(1,1),(1,1),padding='valid')(x)
  x=BatchNormalization(axis=3)(x)

  ### Applying conv operation on x_shortcut 
  x_shortcut=Conv2D(F3,(1,1),(s,s),padding='valid')(x_shortcut)
  x_shortcut=BatchNormalization(axis=3)(x_shortcut)

  ### Add layer
  x=Add()([x,x_shortcut])
  x=Activation('relu')(x)

  return x


In [26]:
def ResNet50(input_shape=(224,224,3),classes=3):
  """
  Implementation of the ResNet50 architecture
  CONV2D -> BATCHNORM -> RELU ->  MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3 -> CONVBLOCK -> IDBLOCK*5 -> 
  CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FC-LAYER
  """

  x_input = Input(input_shape)
  X=ZeroPadding2D((3,3))(x_input)

  ### stage-1
  X=Conv2D(64,(7,7),(2,2))(X)
  X=BatchNormalization(axis=3)(X)
  X=Activation('relu')(X)
  X=MaxPooling2D((3,3),(2,2))(X)

  ### stage-2
  X=convolutional_block(X,f=3,filters=[64,64,256],s=1)
  X=identity_block(X,3,[64,64,256])
  X=identity_block(X,3,[64,64,256])

  ### stage-3
  X=convolutional_block(X,f=3,filters=[128,128,512],s=2)
  X=identity_block(X,3,[128,128,512])
  X=identity_block(X,3,[128,128,512])
  X=identity_block(X,3,[128,128,512])

  ### stage-4 
  X=convolutional_block(X,f=3,filters=[256,256,1024],s=2)
  X=identity_block(X,3,[256,256,1024])
  X=identity_block(X,3,[256,256,1024])
  X=identity_block(X,3,[256,256,1024])
  X=identity_block(X,3,[256,256,1024])
  X=identity_block(X,3,[256,256,1024])

  ### stage-5 
  X=convolutional_block(X,f=3,filters=[512,512,2048],s=2)
  X=identity_block(X,3,[512,512,2048])
  X=identity_block(X,3,[512,512,2048])
  
  ### AVGPOOL 
  X=AveragePooling2D((2,2),name="avg-pool")(X)

  ### output layer

  X=Flatten()(X)
  X=Dense(classes,activation='softmax',name='fc'+str(classes),kernel_initializer=glorot_uniform(seed=0))(X)

  ### create model 
  model=Model(inputs=x_input,outputs=X,name='ResNet50')

  return model



In [28]:
model=ResNet50(input_shape=(224,224,3))

In [29]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=(['accuracy']))

In [30]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_9 (ZeroPadding2D (None, 230, 230, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
conv2d_477 (Conv2D)             (None, 112, 112, 64) 9472        zero_padding2d_9[0][0]           
__________________________________________________________________________________________________
batch_normalization_477 (BatchN (None, 112, 112, 64) 256         conv2d_477[0][0]                 
___________________________________________________________________________________________